In [3]:
#!/usr/bin/env python3
"""
MFEM için dikdörtgen (1x, 1E) mesh üreticisi.
Kullanıcı aşağıdaki parametreleri girecek:
  - filename: Üretilen mesh dosyasının adı
  - x: x yönündeki uzunluk
  - E_max: enerji eksenindeki maksimum değer
  - dx: x eksenindeki hücre genişliği
  - dE: E eksenindeki hücre genişliği
Hücre sayıları:
  Nx = x / dx, NE = E_max / dE (bu bölümlerin tam sayı olması gerekir)
Kullanım:
  Komut satırından: python generate_mesh.py output.mesh 1.0 1.0 0.1 0.1
  Yetersiz argüman varsa, interaktif olarak girdi alınır.
"""

import sys

def generate_mfem_rect_mesh(filename, x, E_max, dx, dE):
    Nx = int(x / dx)
    NE = int(E_max / dE)
    
    with open(filename, 'w') as f:
        # Dosya başlığı ve boyut bilgisi (2D)
        f.write("MFEM mesh v1.0\n\n")
        f.write("2\n\n")
        
        # Vertex'lerin yazılması
        nverts = (Nx + 1) * (NE + 1)
        f.write(f"{nverts}\n")
        for j in range(NE + 1):
            for i in range(Nx + 1):
                x_coord = i * dx
                E_coord = j * dE
                f.write(f"{x_coord} {E_coord}\n")
        f.write("\n")
        
        # Elemanların (dikdörtgenlerin) yazılması
        nelems = Nx * NE
        f.write(f"{nelems}\n")
        for j in range(NE):
            for i in range(Nx):
                # Vertex indeksleri: alt-sol, alt-sağ, üst-sağ, üst-sol
                v0 = j * (Nx + 1) + i
                v1 = v0 + 1
                v3 = (j + 1) * (Nx + 1) + i
                v2 = v3 + 1
                f.write(f"4 {v0} {v1} {v2} {v3}\n")
        f.write("\n")
        
        # Sınır elemanlarının (boundary edges) yazılması
        nbdry = 2 * NE + 2 * Nx
        f.write(f"{nbdry}\n")
        
        # Sol kenar (x = 0)
        for j in range(NE):
            v0 = j * (Nx + 1)
            v1 = (j + 1) * (Nx + 1)
            f.write(f"1 {v0} {v1}\n")
        # Sağ kenar (x = x)
        for j in range(NE):
            v0 = j * (Nx + 1) + Nx
            v1 = (j + 1) * (Nx + 1) + Nx
            f.write(f"1 {v0} {v1}\n")
        # Alt kenar (E = 0)
        for i in range(Nx):
            v0 = i
            v1 = i + 1
            f.write(f"1 {v0} {v1}\n")
        # Üst kenar (E = E_max)
        for i in range(Nx):
            v0 = NE * (Nx + 1) + i
            v1 = v0 + 1
            f.write(f"1 {v0} {v1}\n")
    
    print(f"Mesh dosyası '{filename}' üretildi: x={x}, E_max={E_max}, dx={dx}, dE={dE}")

if __name__ == '__main__':
    # Komut satırı argümanları kontrol ediliyor. (Jupyter gibi ortamlarda sys.argv içinde ekstra değerler olabilir)
    if len(sys.argv) >= 6:
        try:
            filename = sys.argv[1]
            x = float(sys.argv[2])
            E_max = float(sys.argv[3])
            dx = float(sys.argv[4])
            dE = float(sys.argv[5])
        except ValueError:
            print("Hata: x, E_max, dx ve dE sayısal değerler olmalıdır.")
            sys.exit(1)
    else:
        print("Yetersiz veya hatalı argüman, interaktif giriş yapılıyor.")
        filename = input("Dosya adı (varsayılan: rect.mesh): ") or "rect.mesh"
        try:
            x = float(input("x (varsayılan: 1.0): ") or 1.0)
            E_max = float(input("E_max (varsayılan: 1.0): ") or 1.0)
            dx = float(input("dx (varsayılan: 0.1): ") or 0.1)
            dE = float(input("dE (varsayılan: 0.1): ") or 0.1)
        except ValueError:
            print("Girdi hatası: Lütfen sayısal değerler giriniz.")
            sys.exit(1)
    
    # x/dx ve E_max/dE tam sayı olmalı
    if x/dx != int(x/dx) or E_max/dE != int(E_max/dE):
        print("Hata: x/dx ve E_max/dE değerleri tam sayı olmalıdır.")
        sys.exit(1)
    
    generate_mfem_rect_mesh(filename, x, E_max, dx, dE)


Yetersiz veya hatalı argüman, interaktif giriş yapılıyor.


Dosya adı (varsayılan: rect.mesh):  dslkm.mesh
x (varsayılan: 1.0):  1
E_max (varsayılan: 1.0):  10
dx (varsayılan: 0.1):  0.1
dE (varsayılan: 0.1):  1


Mesh dosyası 'dslkm.mesh' üretildi: x=1.0, E_max=10.0, dx=0.1, dE=1.0
